In [192]:
import pytorch_lightning as pl
import numpy as np
import torchaudio
import torch
import torch.optim
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoderLayer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from utils import WaveNet, AudioMELSpectogramDataset #, ContentLoss, StyleLoss

In [193]:
torch.set_float32_matmul_precision('medium')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [194]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# ``nn.Sequential``
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize ``img``
        return (img - self.mean) / self.std

def CalcContentLoss(gen_feat,orig_feat):
    #calculating the content loss of each layer by calculating the MSE between the content and generated features and adding it to content loss
    content_l=torch.mean((gen_feat-orig_feat)**2)
    return content_l

def CalcStyleLoss(gen,style):
    #Calculating the gram matrix for the style and the generated image
    channel,height,width=gen.shape
    G = torch.mm(gen.view(channel,height*width),gen.view(channel,height*width).t())
    # print('G', G.max(), G.min())
    A = torch.mm(style.view(channel,height*width),style.view(channel,height*width).t())
    # print('A', A.max(), A.min())     
    #Calcultating the style loss of each layer by calculating the MSE between the gram matrix of the style image and the generated image and adding it to style loss
    style_l=torch.mean((G-A)**2)
    return style_l

def containsInf(x, text):
    if torch.isinf(x).any():
        print(f"La imagen {text} contiene valores infinitos (inf).")

In [195]:
class StyleAutoEncoder(nn.Module):

    def __init__(self, in_channels, shared_channels, out_channels, kernel_size, dilation, nhead=8, dropout=0.2, dim_feedforward=1024):
        super(StyleAutoEncoder, self).__init__()
        self.sharedEncoder = WaveNet(in_channels, shared_channels, kernel_size, dilation, dropout)
        self.speechEncoder = WaveNet(shared_channels, out_channels, kernel_size, dilation, dropout)
        self.styleEncoder = WaveNet(shared_channels, out_channels, kernel_size, dilation, dropout)
        self.style_encoder_attention = TransformerEncoderLayer(
            d_model=out_channels,
            nhead=8,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.speech_encoder_attention = TransformerEncoderLayer(
            d_model=out_channels,
            nhead=8,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.decoder_attention = TransformerDecoderLayer(
            d_model=out_channels*2,
            nhead=8,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.decoder = WaveNet(out_channels*2, in_channels, kernel_size, dilation, dropout)

    def forward(self, speech, style):

        # print('speech', speech.shape)
        # print('style', style.shape)

        style = self.sharedEncoder(style)
        speech = self.sharedEncoder(speech) 

        style = self.styleEncoder(style)
        speech = self.speechEncoder(speech)
        
        style = style.permute(2, 0, 1)  # Reordenamos las dimensiones para que sea compatible con la capa de atención
        style_encoder = self.style_encoder_attention(style)
        style = style.permute(1, 2, 0)
        style_encoder = speech.permute(0, 1, 2)

        speech = speech.permute(2, 0, 1)
        speech_encoder = self.speech_encoder_attention(speech)
        speech = speech.permute(1, 2, 0)
        speech_encoder = speech.permute(0, 1, 2)
        
        # print('speech', speech.shape)
        # print('style', style.shape)
        out = torch.cat((speech, style), dim=1)
        # print('speech_encoder', speech_encoder.shape)
        # print('style_encoder', style_encoder.shape)
        out_encoder = torch.cat((speech_encoder, style_encoder), dim=1)
        
        out = out.permute(2, 0, 1) 
        out_encoder = out.permute(0, 1, 2) 

        # print('out', out.shape)
        # print('out_encoder', out_encoder.shape)

        x = self.decoder_attention(out, out_encoder)
        x = x.permute(1, 2, 0) 

        # print('x', x.shape)

        x = self.decoder(x)
        # x = x.unsqueeze(1)
        # print('x', x.shape)

        return x #.squeeze(0)
 
class MelSpecVCAutoencoderModule(pl.LightningModule):

    def __init__(self, in_channels, shared_channels, out_channels, kernel_size, dilation, device, sample_rate, dropout=0.2):
        super(MelSpecVCAutoencoderModule, self).__init__()
        self.encoder_generator = StyleAutoEncoder(in_channels, shared_channels, out_channels, kernel_size, dilation, dropout)
        self.todevice = device
        self.speech_weight = 1
        self.style_weight = 100

    def forward(self, speech, style):
        y = self.encoder_generator(speech, style)
        return y

    def debug(self, loss_speech, loss_style, loss):
        # print('loss_speech', loss_speech.item())
        # print('loss_style', loss_style.item())
        # print('shape', loss.shape)
        # print('loss', loss.item())
        pass

    def apply_loss(self, batch):
        speech, style = batch
        if speech.ndim == 2: speech = speech.unsqueeze(1)
        if style.ndim == 2: style = style.unsqueeze(1)
        y = self(speech, style)
        loss_speech = CalcContentLoss(y, speech) * self.speech_weight
        loss_style = CalcStyleLoss(y, style) * self.style_weight
        containsInf(style, 'style')
        containsInf(loss_style, 'loss_style')
        containsInf(loss_speech, 'loss_speech')
        containsInf(y, 'y')
        print(y.shape, style.shape, speech.shape)
        loss = loss_speech + loss_style
        containsInf(loss, 'loss')
        self.debug(loss_speech, loss_style, loss)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.apply_loss(batch)
        self.log('train_loss', loss.item())
        return loss 
    
    def validation_step(self, batch, batch_idx):
        loss = self.apply_loss(batch)
        self.log('val_loss', loss.item())
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

In [196]:
batch_size = 16
sample_rate = 16000
div_ratio = 0.8
n_fft = 800
audio_length = 3
in_channels, shared_channels, out_channels, kernel_size, dilation = 128, 64, 32, 3, 12
metadata = pd.read_csv('metadata.csv')
dataset = AudioMELSpectogramDataset(metadata, device, sample_rate=sample_rate, audio_length=audio_length, n_fft=n_fft)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(len(dataset) * div_ratio), len(dataset) - int(len(dataset) * div_ratio)])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=True)
model = MelSpecVCAutoencoderModule(in_channels, shared_channels, out_channels, kernel_size, dilation, device=device, sample_rate=sample_rate)
model = model.to(device)
model = model.to(torch.float64)

c:\Users\David Arcos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchaudio\functional\functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [197]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='./checkpoints',
    filename='{epoch}-{val_loss:.2f}-{val_r2:.2f}',
    save_top_k=1,
    monitor='val_loss',
    every_n_epochs =1,
    mode='min'
)

trainer = pl.Trainer(
    accelerator='gpu',
    max_epochs=10,
    benchmark=True,
    # deterministic=True,
    precision=16,
    accumulate_grad_batches=6,
    callbacks=[
        checkpoint_callback,
        pl.callbacks.LearningRateMonitor(logging_interval='step')
    ]
)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [198]:
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params
-------------------------------------------------------
0 | encoder_generator | StyleAutoEncoder | 2.3 M 
-------------------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
4.506     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\David Arcos\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


speech torch.Size([16, 128, 241])
style torch.Size([16, 128, 241])
speech torch.Size([16, 32, 241])
style torch.Size([16, 32, 241])
speech_encoder torch.Size([16, 32, 241])
style_encoder torch.Size([16, 32, 241])
torch.Size([16, 128, 241]) torch.Size([16, 128, 241]) torch.Size([16, 128, 241])
speech torch.Size([16, 128, 241])
style torch.Size([16, 128, 241])
speech torch.Size([16, 32, 241])
style torch.Size([16, 32, 241])
speech_encoder torch.Size([16, 32, 241])
style_encoder torch.Size([16, 32, 241])
torch.Size([16, 128, 241]) torch.Size([16, 128, 241]) torch.Size([16, 128, 241])


c:\Users\David Arcos\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

speech torch.Size([16, 128, 241])
style torch.Size([16, 128, 241])
speech torch.Size([16, 32, 241])
style torch.Size([16, 32, 241])
speech_encoder torch.Size([16, 32, 241])
style_encoder torch.Size([16, 32, 241])
torch.Size([16, 128, 241]) torch.Size([16, 128, 241]) torch.Size([16, 128, 241])
speech torch.Size([16, 128, 241])
style torch.Size([16, 128, 241])
speech torch.Size([16, 32, 241])
style torch.Size([16, 32, 241])
speech_encoder torch.Size([16, 32, 241])
style_encoder torch.Size([16, 32, 241])
torch.Size([16, 128, 241]) torch.Size([16, 128, 241]) torch.Size([16, 128, 241])
speech torch.Size([16, 128, 241])
style torch.Size([16, 128, 241])
speech torch.Size([16, 32, 241])
style torch.Size([16, 32, 241])
speech_encoder torch.Size([16, 32, 241])
style_encoder torch.Size([16, 32, 241])
torch.Size([16, 128, 241]) torch.Size([16, 128, 241]) torch.Size([16, 128, 241])
speech torch.Size([16, 128, 241])
style torch.Size([16, 128, 241])
speech torch.Size([16, 32, 241])
style torch.Size([

c:\Users\David Arcos\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
random_speech = train_dataset[120]
mel_spectrogram = random_speech[0]
style_spec = random_speech[1]

# Convert mel spectrogram to linear scale
mel_to_linear = torchaudio.transforms.InverseMelScale(n_stft=800)
linear_spectrogram = mel_to_linear(mel_spectrogram)

# Convert linear spectrogram to waveform
griffin_lim = torchaudio.transforms.GriffinLim()
waveform = griffin_lim(linear_spectrogram)

# Plot mel spectrogram
plt.figure(figsize=(10, 4))
plt.imshow(mel_spectrogram.log2(), cmap='inferno', origin='lower', aspect='auto')

# Set frequency axis ticks
num_ticks = 10
freq_bins = np.linspace(0, mel_spectrogram.shape[0], num_ticks)
hz_ticks = torchaudio.transforms.MelScale().mel_to_hz(torch.tensor(freq_bins))
plt.yticks(freq_bins, ["{:.0f}".format(hz) for hz in hz_ticks])

plt.colorbar(format='%+2.0f dB')
plt.xlabel('Time')
plt.ylabel('Frequency (Hz)')
plt.title('Mel Spectrogram')
plt.show()

# display(ipd.Audio(waveform.cpu().numpy(), rate=sample_rate))

plt.show()